
# Real-Time Sign Language Detection — Optimized TensorFlow Version

This version integrates your existing TensorFlow/Keras model (`model.h5`) into an optimized real-time detection pipeline with:
- Threaded webcam capture to reduce frame delay
- A background inference thread for smoother UI performance
- FPS overlay display

Press **Q** in the video window to stop the program.


In [10]:

import cv2
import threading
import queue
import numpy as np
import time
import tensorflow as tf
import os


In [11]:

# Threaded video capture for reduced latency
class VideoCaptureThread:
    def __init__(self, src=0, width=None, height=None):
        self.cap = cv2.VideoCapture(src)
        if width:
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, int(width))
        if height:
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, int(height))
        self.q = queue.Queue(maxsize=1)
        self.running = False
        self.thread = threading.Thread(target=self._reader, daemon=True)

    def start(self):
        self.running = True
        self.thread.start()
        return self

    def _reader(self):
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()
                except queue.Empty:
                    pass
            self.q.put(frame)

    def read(self, timeout=1.0):
        try:
            return True, self.q.get(timeout=timeout)
        except queue.Empty:
            return False, None

    def stop(self):
        self.running = False
        self.thread.join(timeout=1.0)
        self.cap.release()


In [12]:

# Background worker that performs inference asynchronously
class InferenceWorker:
    def __init__(self):
        self.model = None
        self.in_q = queue.Queue(maxsize=2)
        self.out_q = queue.Queue(maxsize=2)
        self.running = False
        self.thread = threading.Thread(target=self._runner, daemon=True)

    def start(self):
        self.running = True
        self.thread.start()
        return self

    def load_model(self):
        print("Loading TensorFlow model...")
        self.model = tf.keras.models.load_model('model.h5', compile=False)
        print("Model loaded successfully.")

    def _runner(self):
        try:
            self.load_model()
        except Exception as e:
            print("Error loading model:", e)
            self.running = False
            return

        while self.running:
            try:
                frame = self.in_q.get(timeout=0.5)
            except queue.Empty:
                continue

            try:
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224)) / 255.0
                pred = self.model.predict(np.expand_dims(img, axis=0))
                self.out_q.put(pred)
            except Exception as e:
                print("Inference error:", e)

    def submit(self, frame):
        try:
            self.in_q.put_nowait(frame)
            return True
        except queue.Full:
            return False

    def get_result(self, timeout=0.0):
        try:
            return True, self.out_q.get(timeout=timeout)
        except queue.Empty:
            return False, None

    def stop(self):
        self.running = False
        self.thread.join(timeout=1.0)


In [13]:

def main():
    cap = VideoCaptureThread(0, width=640, height=480).start()
    worker = InferenceWorker().start()

    try:
        last_time = time.time()
        while True:
            ret, frame = cap.read()
            if not ret:
                continue

            worker.submit(frame)
            has_res, res = worker.get_result()

            now = time.time()
            fps = 1.0 / (now - last_time)
            last_time = now

            label = "Predicting..." if not has_res else f"Pred: {np.argmax(res)}"
            cv2.putText(frame, f"{label} | FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.imshow("Real-Time Sign Language Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        pass
    finally:
        cap.stop()
        worker.stop()
        cv2.destroyAllWindows()

# Uncomment to run (only works with camera access)
main()


Loading TensorFlow model...
Error loading model: Unrecognized keyword arguments passed to LSTM: {'time_major': False}


ZeroDivisionError: float division by zero


### Notes:
- Ensure `model.h5` is in the same directory as this notebook.
- The input size `(224, 224)` may be changed to match your model’s training dimensions.
- Prediction result (`np.argmax(res)`) assumes categorical output. Adjust if your model differs.
